# Carga y descripción general de los datos

En esta sección, se cargamos el conjunto de datos y  se analiza brevemente su estructura. El conjunto de datos contiene diversas características sobre vehículos eléctricos, incluyendo métricas de rendimiento y especificaciones de las baterías.

In [ ]:
# Cargar el conjunto de datos
data_path = 'electric_vehicles_spec_2025.csv.csv'
df = pd.read_csv(data_path, encoding='EUC-KR')

# Mostrar las primeras filas del conjunto de datos
print('Conjunto de datos cargado con la forma:', df.shape)
df.head()

# Limpieza y preprocesamiento de datos

Antes de comenzar el análisis, limpiamos los datos. Comprobamos si hay valores faltantes, convertimos los tipos de datos si es necesario y preparamos nuestras características.


In [ ]:
# Comprobar valores faltantes
print('Valores faltantes en cada columna:')
print(df.isnull().sum())

# Convertir las columnas a los tipos de datos apropiados si es necesario (ejemplo de conversión para columnas numéricas)
numeric_columns = ['top_speed_kmh', 'battery_capacity_kWh', 'number_of_cells', 'torque_nm', 

'efficiency_wh_per_km', 'range_km', 'acceleration_0_100_s', 

'fast_charging_power_kw_dc', 'towing_capacity_kg', 'seats', 

'length_mm', 'width_mm', 'height_mm']

for col in numeric_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Eliminar filas con valores objetivo faltantes en caso de desarrollo de predictores
df = df.dropna(subset=['acceleration_0_100_s'])

# Para simplificar, completar los valores numéricos faltantes restantes con la mediana y los categóricos con la moda
for col in df.columns:
    if df[col].dtype in ['float64', 'int64']:
        df[col].fillna(df[col].median(), inplace=True)
    else:
        df[col].fillna(df[col].mode()[0], inplace=True)
        
print('Forma de los datos después de la limpieza:', df.shape)

#  Análisis Exploratorio de Datos



In [ ]:
# Dataframe numérico para el mapa de calor de correlación (solo si hay al menos 4 columnas numéricas)
numeric_df = df.select_dtypes(include=[np.number])
if numeric_df.shape[1] >= 4:
    plt.figure(figsize=(12, 10))
    corr = numeric_df.corr()
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
    plt.title('Mapa de calor de correlación')
    plt.show()
else:
    print('No hay suficientes columnas numéricas para un mapa de calor de correlación.')

# Histograma para las características numéricas clave
plt.figure(figsize=(14, 6))
sns.histplot(df['battery_capacity_kWh'], kde=True, color='skyblue')
plt.title('Distribución de la capacidad de la batería (kWh)')
plt.xlabel('Capacidad de la batería (kWh)')
plt.show()

# Gráfico de pares para visualizar las relaciones entre varias variables numéricas
subset_columns = ['top_speed_kmh', 'battery_capacity_kWh', 'range_km', 'acceleration_0_100_s']
sns.pairplot(df[subset_columns])
plt.suptitle('Gráfico de pares de características numéricas seleccionadas', y=1.02)
plt.show()

# Gráfico de conteo para una variable categórica (p. ej., transmisión)
plt.figure(figsize=(8, 4))
sns.countplot(x='drivetrain', data=df, paleta='viridis')
plt.title('Cantidad de vehículos por sistema de transmisión')
plt.xlabel('Sistema de transmisión')
plt.ylabel('Cantidad')
plt.xticks(rotation=45)
plt.show()

# Modelo predictivo

para predecir el tiempo de aceleración de 0 a 100 km/h (en segundos) de estos vehículos eléctricos mediante un modelo de regresión lineal. 

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score

# Define features and target variable. Excluir la variable objetivo y posiblemente otras características irrelevantes

predictor_features = ['top_speed_kmh', 'battery_capacity_kWh', 'number_of_cells', 'torque_nm', 

'efficiency_wh_per_km', 'range_km', 'fast_charging_power_kw_dc', 

'towing_capacity_kg', 'seats', 'length_mm', 'width_mm', 'height_mm']

X = df[predictor_features]
y = df['acceleration_0_100_s']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo
model = LinearRegression()
model.fit(X_train, y_train)
# Realizar predicciones sobre los datos de prueba
y_pred = model.predict(X_test)
# Evaluar el modelo usando el coeficiente de determinación (R²)
r2 = r2_score(y_test, y_pred)
print(f"Calibración R² del modelo de regresión lineal: {r2:.3f}")

# Nota: Si bien el R² proporciona información sobre la varianza explicada, puede ser necesario un análisis adicional con los residuos.

# Graficar valores reales vs. valores predichos
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.7, color='teal')
plt.xlabel('Aceleración real (0-100 s)')
plt.ylabel('Aceleración predicha (0-100 s)')
plt.title('Tiempos de aceleración reales vs. predichos')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.show()
